### IMPORTS

In [1]:
import sys, os
sys.path.append(os.path.join(sys.path[0],'Pythons'))

In [2]:
from Carga_Descarga import *
from CRM_Central import *

In [3]:
pd.set_option('display.float_format', lambda x: '%.5f' % x)

### CONSTANTES

In [4]:
# Fechas
today = datetime.date.today()
idx = (today.weekday() + 1) % 7
fin = today - datetime.timedelta(days=idx)
start = str(fin - datetime.timedelta(6*7-1))
start_tc = str(fin - datetime.timedelta(6*7-1) - relativedelta(months=12))
if today.day == 1:
    end = str(today - relativedelta(days=1))
else:
    end = str(today + relativedelta(months=1) - relativedelta(days=(today + relativedelta(months=1)).day))

In [5]:
# Fechas Weekly
today = datetime.date.today()
idx = (today.weekday() + 1) % 7
fin = today - datetime.timedelta(days=idx)
inicio = fin - datetime.timedelta(1*7-1)
fin = str(fin)
inicio = str(inicio)

### GOOGLE SHEETS

In [6]:
# Descargo los filtros de Campañas
fc = descarga('1d4cMi-V04c80Dpen4S70Q7zYqqho3ZBIVctlRIBLwiw','Filtrar')
# Mando a mayusculas
fc[fc.columns[0]] = fc[fc.columns[0]].str.upper()
fc = fc[fc.columns[0]].tolist()

Descarga Correcta!


### FUNCIONES

In [7]:
def filtrar_cam(i):
    val = 'No'
    for j in fc:
        if j in i:
            val = 'Si'
            break
    return val        

### QUERIES

In [17]:
q_crudo = '''WITH coupons_table AS (
    SELECT cn.country_name AS country,
           tc.talon_campaign_id AS campaign_id,
           tc.coupon_id AS coupon_id,
           o.registered_date AS fecha,
           CASE WHEN bi.payment_mode = 'TOTAL_AMOUNT' THEN 'Si' ELSE 'No' END AS payment_shopper,
           COUNT(DISTINCT o.order_id) AS redeemed,
           COUNT(DISTINCT CASE WHEN o.order_status = 'CONFIRMED' THEN o.order_id ELSE NULL END) AS redeemed_confirmed,
           COUNT(DISTINCT CASE WHEN o.is_acquisition = 1 THEN o.order_id ELSE NULL END) AS acquisitions,
           SUM(tc.coupon_used_amount) AS used_amount,
           SUM(tc.coupon_used_amount / ce.rate_us) AS used_amount_usd,
           SUM(CASE WHEN o.order_status = 'CONFIRMED' THEN tc.coupon_used_amount ELSE 0 END) AS used_amount_confirmed,
           SUM(CASE WHEN o.order_status = 'CONFIRMED' THEN tc.coupon_used_amount / ce.rate_us ELSE 0 END) AS used_amount_confirmed_usd
    FROM `peya-bi-tools-pro.il_core.fact_orders` AS o
    INNER JOIN `peya-bi-tools-pro.il_growth.fact_talon_coupons` AS tc ON o.order_id = tc.order_id
    LEFT JOIN `peya-bi-tools-pro.il_core.dim_partner` AS p ON o.restaurant.id = p.partner_id
    LEFT JOIN `peya-bi-tools-pro.il_core.dim_billing_info` AS bi ON p.billingInfo.billing_info_id = bi.billing_info_id
    LEFT JOIN `peya-bi-tools-pro.il_core.dim_country` AS cn ON tc.coupon_country_id = cn.country_id
    LEFT JOIN `peya-bi-tools-pro.il_core.dim_currency_exchange` AS ce ON cn.currency_id = ce.currency_id AND DATE_TRUNC(o.registered_date,MONTH) = ce.currency_exchange_date
    WHERE o.registered_date BETWEEN DATE('{0}') AND DATE('{1}')
          AND DATE(tc.coupon_created_at) >= DATE('{2}')
    GROUP BY 1,2,3,4,5)
SELECT ct.fecha AS Fecha,
       ct.country AS Country,
       tca.talon_campaign_name AS Campaign,
       ct.payment_shopper AS Payment_Shopper,
       SUM(ct.redeemed) AS Redeemed,
       SUM(ct.redeemed_confirmed) AS Redeemed_Confirmed,
       SUM(ct.acquisitions) AS Acq,
       SUM(ct.used_amount) AS Used_Amount,
       SUM(ct.used_amount_usd) AS Used_Amount_USD,
       SUM(ct.used_amount_confirmed) AS Used_Amount_Confirmed,
       SUM(ct.used_amount_confirmed_usd) AS Used_Amount_Confirmed_USD
FROM coupons_table AS ct
LEFT JOIN `peya-bi-tools-pro.il_growth.dim_talon_campaigns` AS tca ON ct.campaign_id = tca.talon_campaign_id
GROUP BY 1,2,3,4'''.format(start,end,start_tc)

q_wallet = '''WITH wallet_table AS (
    SELECT wa.order_id AS order_id,
           d.campaignCode AS campaign,
           d.amount AS amount
    FROM `peya-bi-tools-pro.il_wallet.fact_wallet_attributions` AS wa,
    UNNEST (attributionDetails) AS d
    WHERE wa.operation = 'Purchase'),
    coupons_table AS (
    SELECT cn.country_name AS country,
           o.registered_date AS fecha,
           wa.campaign AS campaign,
           CASE WHEN bi.payment_mode = 'TOTAL_AMOUNT' THEN 'Si' ELSE 'No' END AS payment_shopper,
           COUNT(DISTINCT o.order_id) AS redeemed,
           COUNT(DISTINCT CASE WHEN o.order_status = 'CONFIRMED' THEN o.order_id ELSE NULL END) AS redeemed_confirmed,
           COUNT(DISTINCT CASE WHEN o.is_acquisition = 1 THEN o.order_id ELSE NULL END) AS acquisitions,
           SUM(CASE WHEN o.order_status = 'CONFIRMED' AND o.registered_date < DATE('2020-06-09') THEN w.amount
                    WHEN o.order_status = 'CONFIRMED' AND o.registered_date >= DATE('2020-06-09') THEN wa.amount
                    ELSE 0 END) AS used_amount_confirmed,
           SUM(CASE WHEN o.order_status = 'CONFIRMED' AND o.registered_date < DATE('2020-06-09') THEN w.amount / ce.rate_us
                    WHEN o.order_status = 'CONFIRMED' AND o.registered_date >= DATE('2020-06-09') THEN wa.amount / ce.rate_us
                    ELSE 0 END) AS used_amount_confirmed_usd,
           SUM(CASE WHEN o.registered_date < DATE('2020-06-09') THEN w.amount
                    WHEN o.registered_date >= DATE('2020-06-09') THEN wa.amount
                    ELSE 0 END) AS used_amount,
           SUM(CASE WHEN o.registered_date < DATE('2020-06-09') THEN w.amount / ce.rate_us
                    WHEN o.registered_date >= DATE('2020-06-09') THEN wa.amount / ce.rate_us
                    ELSE 0 END) AS used_amount_usd
    FROM `peya-bi-tools-pro.il_core.fact_orders` AS o
    INNER JOIN `peya-bi-tools-pro.il_core.fact_order_funding` AS w ON o.order_id = w.order_id AND w.type IN (25,35)
    LEFT JOIN `peya-bi-tools-pro.il_core.dim_partner` AS p ON o.restaurant.id = p.partner_id
    LEFT JOIN `peya-bi-tools-pro.il_core.dim_billing_info` AS bi ON p.billingInfo.billing_info_id = bi.billing_info_id
    LEFT JOIN wallet_table AS wa ON o.order_id = wa.order_id
    LEFT JOIN `peya-bi-tools-pro.il_core.dim_country` AS cn ON o.country_id = cn.country_id
    LEFT JOIN `peya-bi-tools-pro.il_core.dim_currency_exchange` AS ce ON cn.currency_id = ce.currency_id AND DATE_TRUNC(o.registered_date,MONTH) = ce.currency_exchange_date
    WHERE o.registered_date BETWEEN DATE('{0}') AND DATE('{1}')
    GROUP BY 1,2,3,4)
SELECT ct.fecha AS Fecha,
       ct.country AS Country,
       ct.campaign AS Campaign,
       ct.payment_shopper AS Payment_Shopper,
       SUM(ct.redeemed) AS Redeemed,
       SUM(ct.redeemed_confirmed) AS Redeemed_Confirmed,
       SUM(ct.acquisitions) AS Acq,
       SUM(ct.used_amount) AS Used_Amount,
       SUM(ct.used_amount_usd) AS Used_Amount_USD,
       SUM(ct.used_amount_confirmed) AS Used_Amount_Confirmed,
       SUM(ct.used_amount_confirmed_usd) AS Used_Amount_Confirmed_USD
FROM coupons_table AS ct
GROUP BY 1,2,3,4'''.format(start,end)

q_ord = '''SELECT cn.country_name AS Country,
       o.registered_date AS Fecha,
       COUNT(DISTINCT o.order_id) AS Orders,
       COUNT(DISTINCT CASE WHEN o.is_acquisition = 1 THEN o.order_id ELSE NULL END) AS Acq,
       COUNT(DISTINCT o.user.id) AS Active_Users,
FROM `peya-bi-tools-pro.il_core.fact_orders` AS o
LEFT JOIN `peya-bi-tools-pro.il_core.dim_country` AS cn ON o.country_id = cn.country_id
WHERE o.registered_date BETWEEN DATE('{0}') AND DATE('{1}')
      AND o.order_status = 'CONFIRMED'
GROUP BY 1,2'''.format(start,end)

In [9]:
# Descargo la data
bq_crudo = pd.io.gbq.read_gbq(q_crudo, project_id='peya-argentina', dialect='standard')

Downloading: 100%|███████████████████████████████████████████████████████████| 32327/32327 [00:07<00:00, 4180.77rows/s]


In [10]:
bq_wallet = pd.io.gbq.read_gbq(q_wallet, project_id='peya-argentina', dialect='standard')

Downloading: 100%|███████████████████████████████████████████████████████████| 99703/99703 [00:20<00:00, 4791.22rows/s]


In [18]:
bq_ord = pd.io.gbq.read_gbq(q_ord, project_id='peya-argentina', dialect='standard')

Downloading: 100%|███████████████████████████████████████████████████████████████| 664/664 [00:00<00:00, 1571.72rows/s]


In [29]:
# Copio las bases
crudo = bq_crudo.copy()
wallet = bq_wallet.copy()
orders = bq_ord.copy()

### TRABAJO

#### FORMATEO ORDERS

In [30]:
# Doy formato a las columnas
cols_str = ['Fecha','Country']
cols_float = [i for i in orders.columns if i not in cols_str]
orders[cols_float] = orders[cols_float].astype(float)
orders[cols_str] = orders[cols_str].apply(lambda x: x.astype(str).str.upper())
# Doy formato a la fecha
orders['Fecha'] = pd.to_datetime(orders['Fecha'], format='%Y-%m-%d').dt.strftime('%Y-%m-%d')
orders['Month'] = pd.to_datetime(orders['Fecha'], format='%Y-%m').dt.strftime('%Y-%m')
orders['Week'] = pd.to_datetime(orders['Fecha'], format='%Y-%m-%d').dt.strftime('%V')
# Marco las de la ultima semana completa
orders.loc[(orders['Fecha'] >= inicio)&(orders['Fecha'] <= fin),'Week Type'] = 'Last Complete Week'
orders.loc[orders['Fecha'] > fin,'Week Type'] = 'Current Week'
orders.loc[orders['Week Type'].isna(),'Week Type'] = 'Week Before'

In [35]:
# Creo una PT sin fecha
index = ['Country','Month','Week','Week Type']
values = ['Orders','Acq','Active_Users']
pt_orders = orders.pivot_table(index=index,values=values,aggfunc='sum',fill_value=0).reset_index()

In [36]:
# Ordeno la PT Final
pt_orders.sort_values(by=['Month','Week','Country'],inplace=True,ascending=True)
# Marco las semanas finalizadas
pt_orders['Week Finished'] = pt_orders['Week Type'].apply(lambda x: 'Si' if x != 'Current Week' else 'No')

#### FORMATEO CRUDO

In [12]:
# Doy formato a las columnas
cols_str = ['Fecha','Country','Campaign','Payment_Shopper']
cols_float = [i for i in crudo.columns if i not in cols_str]
crudo[cols_float] = crudo[cols_float].astype(float)
crudo[cols_str] = crudo[cols_str].apply(lambda x: x.astype(str).str.upper())
# Saco los espacios de los nombres de campañas
crudo['Campaign'] = crudo['Campaign'].str.replace(' ', '')
# Doy formato a la fecha
crudo['Fecha'] = pd.to_datetime(crudo['Fecha'], format='%Y-%m-%d').dt.strftime('%Y-%m-%d')
crudo['Month'] = pd.to_datetime(crudo['Fecha'], format='%Y-%m').dt.strftime('%Y-%m')
crudo['Week'] = pd.to_datetime(crudo['Fecha'], format='%Y-%m-%d').dt.strftime('%V')
# Marco las de la ultima semana completa
crudo.loc[(crudo['Fecha'] >= inicio)&(crudo['Fecha'] <= fin),'Week Type'] = 'Last Complete Week'
crudo.loc[crudo['Fecha'] > fin,'Week Type'] = 'Current Week'
crudo.loc[crudo['Week Type'].isna(),'Week Type'] = 'Week Before'

In [13]:
# Creo las columnas para los segmentos
crudo['Budget'] = crudo['Campaign'].apply(lambda x: filtros(x,'BUDGET'))
crudo['Type'] = crudo['Campaign'].apply(lambda x: filtros(x,'TIPOS'))
# Marco las campañas a filtrar
crudo['Filtrar'] = crudo['Campaign'].apply(filtrar_cam)
crudo = crudo[crudo['Filtrar'] == 'No'].copy()

#### FORMATEO WALLET

In [14]:
# Doy formato a las columnas
cols_str = ['Fecha','Country','Campaign','Payment_Shopper']
cols_float = [i for i in wallet.columns if i not in cols_str]
wallet[cols_float] = wallet[cols_float].astype(float)
wallet[cols_str] = wallet[cols_str].apply(lambda x: x.astype(str).str.upper())
# Saco los espacios de los nombres de campañas
wallet['Campaign'] = wallet['Campaign'].str.replace(' ', '')
# Doy formato a la fecha
wallet['Fecha'] = pd.to_datetime(wallet['Fecha'], format='%Y-%m-%d').dt.strftime('%Y-%m-%d')
wallet['Month'] = pd.to_datetime(wallet['Fecha'], format='%Y-%m').dt.strftime('%Y-%m')
wallet['Week'] = pd.to_datetime(wallet['Fecha'], format='%Y-%m-%d').dt.strftime('%V')
# Marco las de la ultima semana completa
wallet.loc[(wallet['Fecha'] >= inicio)&(wallet['Fecha'] <= fin),'Week Type'] = 'Last Complete Week'
wallet.loc[wallet['Fecha'] > fin,'Week Type'] = 'Current Week'
wallet.loc[wallet['Week Type'].isna(),'Week Type'] = 'Week Before'

In [15]:
# Creo las columnas para los segmentos
wallet['Budget'] = wallet['Campaign'].apply(lambda x: filtros(x,'BUDGET'))
wallet['Type'] = 'WALLET'
# Marco las campañas a filtrar
wallet['Filtrar'] = wallet['Campaign'].apply(filtrar_cam)
wallet = wallet[wallet['Filtrar'] == 'No'].copy()

#### FINAL

In [16]:
# Uno ambos DF
final = pd.concat([crudo,wallet]).copy()
# Creo una PT
index = ['Month','Week','Week Type','Country','Payment_Shopper','Type','Budget']
values = ['Redeemed','Redeemed_Confirmed','Acq','Used_Amount','Used_Amount_USD',
          'Used_Amount_Confirmed','Used_Amount_Confirmed_USD']
pt = final.pivot_table(index=index,values=values,aggfunc='sum',fill_value=0).reset_index()
# Coloco el IVA
pt = func_iva(pt)
# Saco el IVA
used = 'Used_Amount_Confirmed'
used_usd = 'Used_Amount_Confirmed_USD'
pt.loc[pt['Payment_Shopper'] == 'NO','Used_Final'] = pt[used] / (1 + pt['IVA']) * pt['Extra']
pt.loc[pt['Payment_Shopper'] == 'SI','Used_Final'] = pt[used]
pt.loc[pt['Payment_Shopper'] == 'NO','Used_USD_Final'] = pt[used_usd] / (1 + pt['IVA']) * pt['Extra']
pt.loc[pt['Payment_Shopper'] == 'SI','Used_USD_Final'] = pt[used_usd]

In [17]:
# Ordeno la PT Final
pt.sort_values(by=['Month','Week','Country'],inplace=True,ascending=True)
# Marco las semanas finalizadas
pt['Week Finished'] = pt['Week Type'].apply(lambda x: 'Si' if x != 'Current Week' else 'No')

### CARGA

In [38]:
carga(pt_orders,'1y3HUUDmt73Z3z6rE_KefkFjySSVNfChQwCm0-aUpoq0','Crudo General')

Carga Correcta!


In [18]:
carga(pt,'1y3HUUDmt73Z3z6rE_KefkFjySSVNfChQwCm0-aUpoq0','Crudo')

Carga Correcta!
